## Packages

In [1]:
import torch.nn as nn
import torch
import torch.optim as optim

from PIL import ImageFile
from torch.utils.data import DataLoader
from torchvision import transforms
from tools import ImageNet, init_weights, init_bias, training_loop

## Configuration

In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

trn_batch_size = 256
val_batch_size = 100
epoch_count = 74
learning_rate = 0.00001
momentum = 0.9
weight_decay = 0.0005
dropout_ratio = 0.5

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Datasets

In [3]:
imgnet_trn_set = ImageNet(transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.RandomCrop((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[1.0, 1.0, 1.0]),
    transforms.ColorJitter((0.3, 0.7), (0.3, 0.7), (0.3, 0.7)),
]), is_train=True)

print("ImgNet set loaded...")

imgnet_trn_loader = DataLoader(
    dataset=imgnet_trn_set,
    batch_size=trn_batch_size,
    shuffle=True,
)

ImgNet set loaded...


## Structure

In [4]:
VGG19 = nn.Sequential(

)

class VGG19(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d( 3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d( 64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.conv_block5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fucn_block6 = nn.Sequential(
            nn.Conv2d( 512, 4096, kernel_size=7),
            nn.ReLU(),
            nn.Dropout(p=dropout_ratio),
            nn.Conv2d(4096, 4096, kernel_size=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_ratio),
            nn.Conv2d(4096, 1000, kernel_size=1),
        )

    def forward(self, x):
        output = self.conv_block1(x)
        output = self.conv_block2(output)
        output = self.conv_block3(output)
        output = self.conv_block4(output)
        output = self.conv_block5(output)
        output = self.fucn_block6(output)

        return output.view(trn_batch_size, -1)

Vgg19 = VGG19()

init_weights(Vgg19, mean=0.0, std=0.01)
init_bias(Vgg19, 0)

Vgg19.to(device)

VGG19(
  (conv_block1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7):

## Train

In [5]:
training_loop(
    n_epochs=epoch_count,
    optimizer=optim.SGD(
        Vgg19.parameters(),
        lr=learning_rate,
        momentum=momentum,
        weight_decay=weight_decay,
    ),
    model=Vgg19,
    loss_fn=nn.CrossEntropyLoss(),
    dev=device,
    loader=imgnet_trn_loader,
)

torch.save(Vgg19.state_dict(), "VggNet_Weights.pt")
print("VggNet parameters saved...")

KeyboardInterrupt: 